# Все подключения

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
from datetime import datetime
import os
from tqdm import tqdm
import time

In [2]:
#DB connection settings:
db_config = {'user': 'aykui.arakelyan',         # Username
                 'pwd': '7%40EG2xc2', # passowrd
                 'host': 'dwh-gp-master01',       # server address
                 'port': 7433,              # port
                 'db': 'ckdata'}             # DB name

    # connection string: 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
    # Connect:
engine = create_engine(connection_string)

In [3]:
folder_path = '/Users/ayaarakelyan/Desktop/ПИ/'
data = pd.read_csv(folder_path+'Алена.csv', header=0, sep=";")

/var/folders/8x/mm0d18r53xgbz3_6347vsms80000gn/T/ipykernel_57269/2597190646.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(folder_path+'Алена.csv', header=0, sep=";")


In [4]:
data[['Начало периода', 'Окончание периода', 'Дата ПИ']] = data[['Начало периода', 'Окончание периода', 'Дата ПИ']].applymap(
    lambda x: datetime.strptime(x, "%d.%m.%Y").strftime("%Y-%m-%d") if isinstance(x, str) else x)

In [5]:
data['key'] = data['Магазин'].astype(str)+'|'+data['Начало периода']+'|'+data['Окончание периода']

In [6]:
data.head()

,Магазин,Начало периода,Окончание периода,Дата ПИ,Товар,key
0,2031,2023-05-07,2023-07-06,2023-06-06,4037662,2031|2023-05-07|2023-07-06
1,2031,2023-08-27,2023-10-26,2023-09-26,3425945,2031|2023-08-27|2023-10-26
2,2031,2023-05-07,2023-07-06,2023-06-06,3418493,2031|2023-05-07|2023-07-06
3,2031,2023-05-07,2023-07-06,2023-06-06,3625733,2031|2023-05-07|2023-07-06
4,2031,2023-08-27,2023-10-26,2023-09-26,4053082,2031|2023-08-27|2023-10-26


In [7]:
# Группируем по столбцам Магазин, Начало периода, Окончание периода и дата ПИ, а затем объединяем значения столбца Товар через запятую
result = data.groupby(['Магазин', 'Начало периода', 'Окончание периода', 'Дата ПИ', 'key'])['Товар'].apply(lambda x: ','.join(map(str, x))).reset_index()
result['Товар'] = result['Товар'].apply(lambda x: f"'{x}'").str.replace(',', "', '")
result.head()
result.shape

(898, 6)

In [30]:
result2 = result[800:]
result2.shape

(98, 6)

In [31]:
result2.shape

(98, 6)

In [32]:
list_key = result2['key'].unique().astype(object)

### Выгрузка из БД

In [33]:
output_folder = '/Users/ayaarakelyan/Desktop/ПИ/Выгрузки/'

In [34]:
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

df_combined = pd.DataFrame()

for element in tqdm(list_key, desc='Processing'):
    start_date = result2.loc[(result2['key'] == element)]['Начало периода'].iloc[0]
    end_date = result2.loc[(result2['key'] == element)]['Окончание периода'].iloc[0]
    inventory_date = result2.loc[(result2['key'] == element)]['Дата ПИ'].iloc[0]
    zavod = result2.loc[(result2['key'] == element)]['Магазин'].iloc[0]
    list_plu2 = result2.loc[(result2['key'] == element)]['Товар'].iloc[0]
    with engine.begin() as conn:
        query = text(f'''SELECT     
    sp.stock_point_id,     
    material_id, 
    material_nm,      
    CASE         
        WHEN sp.stock_point_id = '{zavod}' AND s.cash_register_dt < '{inventory_date}' THEN 'До ПИ'
        WHEN sp.stock_point_id = '{zavod}' AND s.cash_register_dt >= '{inventory_date}' THEN 'После ПИ'    
    END AS period_of_inventory,
    SUM(turnover_vat_rub_amt) rto,     
    SUM(turnover_no_vat_amt) sales,
    SUM(turnover_base_uom_amt) count_of_sales
FROM     GRP_EM.FCT_SALES_D s
LEFT JOIN     GRP_EM.DIM_MATERIAL m ON s.plu_rk = m.material_rk
LEFT JOIN     GRP_EM.DIM_STOCK_POINT sp ON s.store_rk = sp.store_rk AND m.valid_to_dttm = '5999-01-01'
WHERE
    sp.stock_point_id = '{zavod}' AND material_id IN ({list_plu2}) AND s.cash_register_dt BETWEEN '{start_date}' AND '{end_date}' 
GROUP BY 
     sp.stock_point_id, 
     material_id, 
     material_nm, 
     sp.stock_point_id, 
     period_of_inventory ''')
        df = pd.read_sql_query(query, conn)
    df_combined = pd.concat([df_combined, df], ignore_index=True)
    
output_file = os.path.join(output_folder, 'output_data.csv')
df_combined.to_csv(output_file, index=False)

Processing: 100%|██████████████████████████████████████████████████████████| 98/98 [13:13<00:00,  8.10s/it]


In [39]:
output_file_1 = pd.read_csv(output_folder+'output_data_1.csv', header=0, sep=",")
output_file_2 = pd.read_csv(output_folder+'output_data_2.csv', header=0, sep=",")
output_file_3 = pd.read_csv(output_folder+'output_data_3.csv', header=0, sep=",")
output_file_4 = pd.read_csv(output_folder+'output_data_4.csv', header=0, sep=",")
output_file_5 = pd.read_csv(output_folder+'output_data_5.csv', header=0, sep=",")
output_file_6 = pd.read_csv(output_folder+'output_data_6.csv', header=0, sep=",")
output_file_7 = pd.read_csv(output_folder+'output_data_7.csv', header=0, sep=",")

In [43]:
data_output = pd.concat([output_file_1, output_file_2, output_file_3, output_file_4, output_file_5, output_file_6, output_file_7 ], ignore_index=True)
data_output.shape

(918150, 7)

In [44]:
output_file_total = os.path.join(output_folder, 'output_data.csv')
data_output.to_csv(output_file_total, index=False)

In [ ]:
        save_path = os.path.join(output_folder, f'df_{element}.csv')
    df_combined.to_csv(save_path, index=False)